# [Data Cleaning and Preperation](https://wesmckinney.com/book/data-cleaning.html)

Listen, it's natural that you will run into missing data. During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst's time.

And sometimes that Data is missing! How _fun_, but that does mean we need a way to handle it.

thankfully, Panda's has you covered.

# 7.1 Handling Missing Data

all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is sufficient for most real-world use. For data with `float64` dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

We call this a ___sentinel value___: when present, it indicats a missing or _null_ value

In [4]:
import pandas as pd
import numpy as np
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

The `isna` method gives us a Boolean Series with `True` where values are null:

In [5]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that __does not exist__ or __that exists but was not observed (through problems with data collection, for example)__. When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

Python as `None`, and that is treated as NA:

In [6]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data


0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [7]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [8]:
float_data = pd.Series([1, 2, None], dtype='float64')

float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [9]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

pandas has attempted to make working with missing data consistent across data types. Functions like `pandas.isna` abstract away many of the annoying details. Here are some more [Na Handling object methods](https://wesmckinney.com/book/data-cleaning.html#tbl-table_na_method)

## Filtering Out Missing Data

There are a few ways to filter out missing data. While you can use `pandas.isna` and Boolean indexing `dropna` can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [10]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is the same as the following:

In [11]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all NA, or only those rows or columns containing any NAs at all. `dropna` by default drops __any row containing a missing value__:



In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all NA:


In [14]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Keep in mind that these functions return new objects by default and do not modify the contents of the original object.

To drop columns in the same way, pass `axis="columns"`:



In [26]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [27]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep __only rows containing at most a certain number of missing observations__. You can indicate this with the `thresh` argument:



In [28]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df


,0,1,2
0,0.425636,NaN,NaN
1,-0.646057,NaN,NaN
2,-0.846033,NaN,-0.105654
3,-0.423605,NaN,-1.834470
4,1.207579,-0.816933,2.705048
5,0.587507,1.534074,-0.244777
6,0.134532,1.129826,-0.256194


In [29]:
df.dropna()


,0,1,2
4,1.207579,-0.816933,2.705048
5,0.587507,1.534074,-0.244777
6,0.134532,1.129826,-0.256194


In [32]:
df.dropna(thresh=2)

,0,1,2
2,-0.846033,NaN,-0.105654
3,-0.423605,NaN,-1.834470
4,1.207579,-0.816933,2.705048
5,0.587507,1.534074,-0.244777
6,0.134532,1.129826,-0.256194


## Filter in Missing Data
Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:|



In [35]:
df.fillna(0)

,0,1,2
0,0.425636,0.000000,0.000000
1,-0.646057,0.000000,0.000000
2,-0.846033,0.000000,-0.105654
3,-0.423605,0.000000,-1.834470
4,1.207579,-0.816933,2.705048
5,0.587507,1.534074,-0.244777
6,0.134532,1.129826,-0.256194


Calling `fillna` with a dictionary, you can use a different fill value for each column:



In [37]:
df.fillna({1: 0.5, 
           2: 0})

,0,1,2
0,0.425636,0.500000,0.000000
1,-0.646057,0.500000,0.000000
2,-0.846033,0.500000,-0.105654
3,-0.423605,0.500000,-1.834470
4,1.207579,-0.816933,2.705048
5,0.587507,1.534074,-0.244777
6,0.134532,1.129826,-0.256194


The same intrerpolation methods available for reindexing can be used with `fillna`:

In [38]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-0.942691,0.067279,-0.572184
1,-0.706790,0.593234,-0.315014
2,-1.703801,NaN,-2.587198
3,0.793631,NaN,-0.079630
4,-0.316540,NaN,NaN
5,2.006721,NaN,NaN


In [39]:
df.fillna(method="ffill")

,0,1,2
0,-0.942691,0.067279,-0.572184
1,-0.706790,0.593234,-0.315014
2,-1.703801,0.593234,-2.587198
3,0.793631,0.593234,-0.079630
4,-0.316540,0.593234,-0.079630
5,2.006721,0.593234,-0.079630


In [40]:
df.fillna(method="ffill", limit=2)



,0,1,2
0,-0.942691,0.067279,-0.572184
1,-0.706790,0.593234,-0.315014
2,-1.703801,0.593234,-2.587198
3,0.793631,0.593234,-0.079630
4,-0.316540,NaN,-0.079630
5,2.006721,NaN,-0.079630


With `fillna` you can do lots of other things such as simple data imputation using the median or mean statistics:



In [41]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

[`fillna`](https://wesmckinney.com/book/data-cleaning.html#tbl-table_fillna_function) function arguments

# Data Transformation

So far we have been concerned about handling missing data. Filtering, cleaning, and other transformations are another class of important operations.

## Removing Duplications
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [43]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a Boolean Series indicating whether each row is a duplicate (its column values are exactly equal to those in an earlier row) or not:



In [44]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame with rows where the `duplicated` array is `False` filtered out:



In [45]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the `"k1"` column:



In [46]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [47]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:



In [48]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:



In [49]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:



In [50]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The `map` method on a Series (also discussed in [Ch 5.2.5: Function Application and Mapping](https://wesmckinney.com/book/pandas-basics.html#pandas_apply)) accepts a function or dictionary-like object containing a mapping to do the transformation of values:


In [51]:
data["animal"] = data["food"].map(meat_to_animal)
data


,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:



In [52]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map` is a convenient way to perform element-wise transformations and other data cleaning-related operations.



## Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value replacement. As you've already seen, `map` can be used to modify a subset of values in an object, but `replace` provides a simpler and more flexible way to do so. Let’s consider this Series:



In [53]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be sentinel values for missing data. To `replace` these with NA values that pandas understands, we can use replace, producing a new Series:



In [54]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

You can replace multiple values at once by passing a list and then the substitue value:

In [55]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [59]:
data # A reminder of what we are messing with

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [56]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dictionary:

In [60]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

 - The `data.replace` method is distinct from `data.str.replace`, which performs element-wise string substitution. We look at these string methods on Series later in the chapter.

## Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in place without creating a new data structure. Here’s a simple example:



In [61]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

Like a Series, the axis indexes have a `map` method:

In [62]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to the `index` attribute, modifying the DataFrame in place:



In [63]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`:



In [64]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can be used in conjunction with a dictionary-like object, providing new values for a subset of the axis labels:



In [65]:
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you from the chore of copying the DataFrame manually and assigning new values to its `index` and `columns` attributes.

# Discretization and Binning

Continuous data is often discretized or otherwise seperated into "bins" for analysis, say you have a group of people in a study, and you want to group them into discrete age buckets.



In [66]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `pandas.cut`:



In [67]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by `pandas.cut`. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:



In [68]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [71]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [72]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [73]:
pd.value_counts(age_categories)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Note that `pd.value_counts(categories)` are the bin counts for the result of `pandas.cut`.

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing `right=False`:



In [74]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can override the default interval-based bin labeling by passing a list or array to the `labels` option:



In [75]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `pandas.cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:


In [76]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.0051, 0.24], (0.0051, 0.24], (0.47, 0.7], (0.47, 0.7], (0.47, 0.7], ..., (0.24, 0.47], (0.7, 0.93], (0.47, 0.7], (0.7, 0.93], (0.0051, 0.24]]
Length: 20
Categories (4, interval[float64, right]): [(0.0051, 0.24] < (0.24, 0.47] < (0.47, 0.7] < (0.7, 0.93]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `pandas.qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `pandas.cut` will not usually result in each bin having the same number of data points. Since `pandas.qcut` uses sample quantiles instead, you will obtain roughly equally sized bins:

In [77]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-0.013, 0.64], (-0.013, 0.64], (0.64, 2.9], (-0.013, 0.64], (-0.65, -0.013], ..., (0.64, 2.9], (-3.2899999999999996, -0.65], (-0.65, -0.013], (0.64, 2.9], (-3.2899999999999996, -0.65]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.2899999999999996, -0.65] < (-0.65, -0.013] < (-0.013, 0.64] < (0.64, 2.9]]

In [78]:
pd.value_counts(quartiles)

(-3.2899999999999996, -0.65]    250
(-0.65, -0.013]                 250
(-0.013, 0.64]                  250
(0.64, 2.9]                     250
dtype: int64

Similar to `pandas.cut`, you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [80]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.2849999999999997, -1.276]    100
(-1.276, -0.0128]                400
(-0.0128, 1.237]                 400
(1.237, 2.899]                   100
dtype: int64

We’ll return to `pandas.cut` and `pandas.qcut` later in the chapter during our discussion of aggregation and group operations, as these discretization functions are especially useful for quantile and group analysis.

## Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [81]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.013727,0.016124,0.053950,0.018427
std,0.970203,0.992586,0.999442,0.966785
min,-3.595649,-2.845230,-3.759917,-3.334341
25%,-0.655375,-0.650991,-0.605517,-0.591681
50%,-0.028370,-0.004231,0.044667,0.006750
75%,0.706228,0.667222,0.736021,0.717460
max,3.958870,2.848678,3.136542,3.514628


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:



In [82]:
col = data[2]
col[col.abs() > 3]

135   -3.182980
305    3.136542
430   -3.759917
599    3.116168
961    3.012242
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the `any` method on a Boolean DataFrame:



In [83]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
22,-3.103974,-0.671301,0.553752,1.162625
125,-3.008060,0.365666,-1.175830,1.332293
135,0.187068,-0.131771,-3.182980,-0.419307
264,-0.742537,2.352986,0.332697,-3.334341
305,-0.767116,-0.297675,3.136542,0.672741
430,0.439061,-0.481934,-3.759917,-1.316357
496,0.097042,1.521343,-0.180553,-3.071811
521,3.958870,-1.725493,0.198352,-0.952475
599,-1.170202,0.358223,3.116168,1.579575
872,2.583195,1.748294,-0.631133,3.514628


The parentheses around `data.abs() > 3` are necessary in order to call the `any` method on the result of the comparison operation.

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:



In [84]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.013476,0.016124,0.054628,0.018318
std,0.964368,0.992586,0.995446,0.963724
min,-3.000000,-2.845230,-3.000000,-3.000000
25%,-0.655375,-0.650991,-0.605517,-0.591681
50%,-0.028370,-0.004231,0.044667,0.006750
75%,0.706228,0.667222,0.736021,0.717460
max,3.000000,2.848678,3.000000,3.000000


The statement `np.sign(data)` produces 1 and –1 values based on whether the values in `data` are positive or negative:



In [85]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,1.0,-1.0


## Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:



In [86]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [87]:
sampler = np.random.permutation(5)
sampler


array([1, 3, 4, 2, 0])